In [ ]:
# Example of calculating the average cosinesimilarities for one sample
import numpy as np
import torch
torch.cuda.set_device(1)
import matplotlib.pyplot as plt

cls_map = []
cls_map_a = []

for i in range(12):
    b = torch.load("/home/recordattn/cls_attn_tiny{}.pth".format(i),map_location="cpu")
    cls_map_a.append(b)
    b = b.flatten()
    cls_map.append(b)
    
def calculate_cosine_similarity(patch_representations):
    normalized_patches = F.normalize(patch_representations, p=2, dim=1)
    cosine_similarities = torch.mm(normalized_patches, normalized_patches.t()) #([196, 196])
    return cosine_similarities

def average_cosine_similarity(cosine_similarities):
    valid_cosine_similarities = cosine_similarities - torch.eye(cosine_similarities.size(0), device=cosine_similarities.device)
    average_similarity = torch.mean(valid_cosine_similarities)
    return average_similarity.item()

average_cosine_similarities_per_layer = []
for patch_representations in cls_map_a:
    cosine_similarities = calculate_cosine_similarity(patch_representations)
    average_similarity = average_cosine_similarity(cosine_similarities)
    average_cosine_similarities_per_layer.append(average_similarity)

print(average_cosine_similarities_per_layer)

In [ ]:
av_cos_a = np.array(average_cosine_similarities_per_layer)

keep_ratio_min = 0.6
alpha = (1-keep_ratio_min)/(av_cos_a.min()-av_cos_a.max()) 
beta = 1 - alpha*av_cos_a.min()
print(alpha,beta)

keep_ratio = []
for i in range(12):
    temp = alpha*av_cos_a[i] + beta
    keep_ratio.append(temp)

import matplotlib.pyplot as plt    
plt.figure(figsize=(15, 10), dpi=100)
label = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
plt.plot(label, keep_ratio)
plt.show()

print(keep_ratio)